<a href="https://colab.research.google.com/github/Savville/2025-design-week-2/blob/main/ML_on_Structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install openseespy

In [6]:
import openseespy.opensees as ops
import numpy as np
import matplotlib.pyplot as plt

# Clear any existing model
ops.wipe()

# Create ModelBuilder
ops.model('basic', '-ndm', 2, '-ndf', 2)

print("Creating corrected truss model based on the geometry...")

# Define nodes based on careful interpretation of the truss geometry
# Bottom chord nodes (from the coordinates shown)
ops.node(1, 0.0, 0.0)    # (0,0)
ops.node(3, 6.0, 0.0)    # (6,0)
ops.node(5, 12.0, 0.0)   # (12,0)
ops.node(7, 18.0, 0.0)   # (18,0)
ops.node(9, 24.0, 0.0)   # (24,0)
ops.node(11, 30.0, 0.0)  # (30,0)

# Top chord nodes (estimating height from triangular geometry)
height = 4.5  # Estimated truss height
ops.node(2, 3.0, height)   # (3,0) moved up
ops.node(4, 9.0, height)   # (9,0) moved up
ops.node(6, 15.0, height)  # (15,0) moved up
ops.node(8, 21.0, height)  # (21,0) moved up
ops.node(10, 27.0, height) # (27,0) moved up

# Define material properties
E = 200000.0e6  # Young's modulus (Pa) - 200 GPa
A_chord = 0.01   # Cross-sectional area for chord members (m²)
A_web = 0.005    # Cross-sectional area for web members (m²)

# Define material
ops.uniaxialMaterial('Elastic', 1, E)

# Define elements with proper connectivity
element_tag = 1

print("Creating elements...")

# Bottom chord elements (horizontal members at bottom)
bottom_connections = [(1,3), (3,5), (5,7), (7,9), (9,11)]
print("Bottom chord elements:")
for connection in bottom_connections:
    ops.element('Truss', element_tag, connection[0], connection[1], A_chord, 1)
    print(f"  Element {element_tag}: Node {connection[0]} to Node {connection[1]}")
    element_tag += 1

# Top chord elements (horizontal members at top)
top_connections = [(2,4), (4,6), (6,8), (8,10)]
print("Top chord elements:")
for connection in top_connections:
    ops.element('Truss', element_tag, connection[0], connection[1], A_chord, 1)
    print(f"  Element {element_tag}: Node {connection[0]} to Node {connection[1]}")
    element_tag += 1

# Vertical and diagonal web members (creating stable triangular patterns)
web_connections = [
    # Verticals and diagonals to create stable triangulation
    (1, 2),   # Left support to first top node
    (2, 3),   # First triangle diagonal
    (3, 4),   # Vertical-like member
    (4, 5),   # Second triangle diagonal
    (5, 6),   # Vertical-like member
    (6, 7),   # Third triangle diagonal
    (7, 8),   # Vertical-like member
    (8, 9),   # Fourth triangle diagonal
    (9, 10),  # Vertical-like member
    (10, 11), # Right support connection
    # Additional diagonals for stability
    (1, 4),   # Long diagonal
    (3, 6),   # Long diagonal
    (5, 8),   # Long diagonal
    (7, 10),  # Long diagonal
]

print("Web elements:")
for connection in web_connections:
    ops.element('Truss', element_tag, connection[0], connection[1], A_web, 1)
    print(f"  Element {element_tag}: Node {connection[0]} to Node {connection[1]}")
    element_tag += 1

# Apply boundary conditions
print("\nApplying boundary conditions...")
# Pin support at node 1 (left end)
ops.fix(1, 1, 1)  # Fixed in both x and y directions
print("  Node 1: Pinned support (fixed in X and Y)")

# Roller support at node 11 (right end)
ops.fix(11, 0, 1)  # Fixed in y direction, free in x direction
print("  Node 11: Roller support (fixed in Y, free in X)")

# Define loads
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)

# Apply point loads at top chord nodes
load_magnitude = -10000.0  # 10 kN downward
loaded_nodes = [2, 4, 6, 8, 10]
print(f"\nApplying loads ({load_magnitude/1000:.0f} kN downward):")
for node in loaded_nodes:
    ops.load(node, 0.0, load_magnitude)
    print(f"  Node {node}: {load_magnitude/1000:.0f} kN")

# Analysis setup with more robust settings
print("\nSetting up analysis...")
ops.constraints('Transformation')  # Better constraint handler
ops.numberer('RCM')
ops.system('BandGeneral')  # More robust system solver
ops.test('NormDispIncr', 1.0e-6, 50)  # Relaxed tolerance, more iterations
ops.algorithm('Newton')
ops.integrator('LoadControl', 0.1)  # Smaller load steps
ops.analysis('Static')

# Run analysis with load stepping
print("Running static analysis...")
success = True
for step in range(10):  # 10 load steps of 0.1 each = 1.0 total
    result = ops.analyze(1)
    if result != 0:
        print(f"  Analysis failed at step {step+1}")
        success = False
        break
    else:
        print(f"  Load step {step+1}/10 completed")

if success:
    print("Static analysis completed successfully!")

    # Get results
    print("\nStatic Analysis Results:")
    print("========================")

    # Node displacements
    print("\nNode Displacements:")
    print("Node\tX-Disp (mm)\tY-Disp (mm)")
    print("-" * 35)

    all_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    max_disp = 0

    for node in all_nodes:
        disp = ops.nodeDisp(node)
        max_disp = max(max_disp, abs(disp[0]), abs(disp[1]))
        print(f"{node}\t{disp[0]*1000:.3f}\t\t{disp[1]*1000:.3f}")

    # Element forces
    print("\nElement Forces:")
    print("Element\tAxial Force (kN)")
    print("-" * 25)

    max_force = 0
    for elem in range(1, element_tag):
        force = ops.eleForce(elem)
        max_force = max(max_force, abs(force[0]))
        force_kN = force[0] / 1000.0
        status = "Tension" if force[0] > 0 else "Compression" if force[0] < 0 else "Zero"
        print(f"{elem}\t{force_kN:.2f} ({status})")

    # Reactions
    print("\nSupport Reactions:")
    print("Node\tRx (kN)\t\tRy (kN)")
    print("-" * 30)

    total_vertical_reaction = 0
    for node in [1, 11]:
        reaction = ops.nodeReaction(node)
        reaction_x_kN = reaction[0] / 1000.0
        reaction_y_kN = reaction[1] / 1000.0
        total_vertical_reaction += reaction_y_kN
        print(f"{node}\t{reaction_x_kN:.2f}\t\t{reaction_y_kN:.2f}")

    # Check equilibrium
    total_applied_load = len(loaded_nodes) * load_magnitude / 1000.0
    print(f"\nEquilibrium Check:")
    print(f"Total applied load: {total_applied_load:.1f} kN")
    print(f"Total vertical reactions: {total_vertical_reaction:.1f} kN")
    print(f"Difference: {abs(total_applied_load - total_vertical_reaction):.3f} kN")

    if abs(total_applied_load - total_vertical_reaction) < 0.1:
        print("✓ Equilibrium satisfied")
    else:
        print("⚠ Equilibrium not satisfied - check model")

    # Modal Analysis (if static analysis was successful)
    print("\n" + "="*50)
    print("MODAL ANALYSIS - NATURAL FREQUENCIES")
    print("="*50)

    # Add mass for dynamic analysis
    rho = 7850.0  # Steel density (kg/m³)
    volume_per_node = A_chord * 3.0  # Approximate volume per node
    node_mass = rho * volume_per_node

    print(f"Adding masses: {node_mass:.1f} kg per node")
    for node in all_nodes:
        ops.mass(node, node_mass, node_mass)

    # Perform eigenvalue analysis
    num_modes = 6
    eigenvalues = ops.eigen(num_modes)

    if eigenvalues and all(ev > 1e-10 for ev in eigenvalues):
        print(f"\nFirst {num_modes} Natural Frequencies:")
        print("Mode\tFrequency (Hz)\tPeriod (s)")
        print("-" * 35)

        frequencies = []
        for i, eigenval in enumerate(eigenvalues):
            omega = eigenval**0.5
            frequency = omega / (2 * np.pi)
            period = 1.0 / frequency
            frequencies.append(frequency)
            print(f"{i+1}\t{frequency:.3f}\t\t{period:.6f}")

        print(f"\nFrequency Summary:")
        print(f"Fundamental frequency: {frequencies[0]:.3f} Hz")
        print(f"Fundamental period: {1.0/frequencies[0]:.3f} s")

    else:
        print("Modal analysis failed - check structural stability")

else:
    print("Static analysis failed!")
    print("\nPossible issues:")
    print("1. Check element connectivity")
    print("2. Verify boundary conditions")
    print("3. Check for mechanisms in the structure")
    print("4. Review node coordinates")

# Enhanced plotting function
def plot_results():
    if not success:
        print("Cannot plot - analysis failed")
        return

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # Plot 1: Undeformed structure
    ax1.set_title('Truss Geometry')
    ax1.grid(True, alpha=0.3)

    # Plot nodes
    for node in all_nodes:
        coord = ops.nodeCoord(node)
        ax1.plot(coord[0], coord[1], 'ro', markersize=8)
        ax1.text(coord[0], coord[1]+0.3, str(node), ha='center', va='bottom', fontweight='bold')

    # Plot elements
    for elem in range(1, element_tag):
        try:
            nodes = ops.eleNodes(elem)
            coord1 = ops.nodeCoord(nodes[0])
            coord2 = ops.nodeCoord(nodes[1])
            ax1.plot([coord1[0], coord2[0]], [coord1[1], coord2[1]], 'b-', linewidth=2)
        except:
            continue

    # Add support symbols
    coord1 = ops.nodeCoord(1)
    coord11 = ops.nodeCoord(11)
    ax1.plot(coord1[0], coord1[1], 's', markersize=12, color='green', label='Pin Support')
    ax1.plot(coord11[0], coord11[1], '^', markersize=12, color='orange', label='Roller Support')
    ax1.legend()
    ax1.axis('equal')

    # Plot 2: Deformed shape
    ax2.set_title(f'Deformed Shape (Max disp: {max_disp*1000:.2f} mm)')
    ax2.grid(True, alpha=0.3)

    scale_factor = 100 if max_disp > 0 else 1

    for node in all_nodes:
        coord = ops.nodeCoord(node)
        disp = ops.nodeDisp(node)
        # Original
        ax2.plot(coord[0], coord[1], 'ro', markersize=6, alpha=0.5)
        # Deformed
        new_x = coord[0] + disp[0] * scale_factor
        new_y = coord[1] + disp[1] * scale_factor
        ax2.plot(new_x, new_y, 'bo', markersize=6)

    # Plot deformed elements
    for elem in range(1, element_tag):
        try:
            nodes = ops.eleNodes(elem)
            coord1 = ops.nodeCoord(nodes[0])
            coord2 = ops.nodeCoord(nodes[1])
            disp1 = ops.nodeDisp(nodes[0])
            disp2 = ops.nodeDisp(nodes[1])

            new_x1 = coord1[0] + disp1[0] * scale_factor
            new_y1 = coord1[1] + disp1[1] * scale_factor
            new_x2 = coord2[0] + disp2[0] * scale_factor
            new_y2 = coord2[1] + disp2[1] * scale_factor

            ax2.plot([new_x1, new_x2], [new_y1, new_y2], 'b-', linewidth=2)
        except:
            continue

    ax2.axis('equal')

    # Plot 3: Force diagram
    ax3.set_title(f'Axial Forces (Max: {max_force/1000:.1f} kN)')
    ax3.grid(True, alpha=0.3)

    for elem in range(1, element_tag):
        try:
            nodes = ops.eleNodes(elem)
            coord1 = ops.nodeCoord(nodes[0])
            coord2 = ops.nodeCoord(nodes[1])
            force = ops.eleForce(elem)[0] / 1000.0  # Convert to kN

            # Color code: red for tension, blue for compression
            color = 'red' if force > 0 else 'blue'
            linewidth = min(5, max(1, abs(force) / max_force * 5)) if max_force > 0 else 1

            ax3.plot([coord1[0], coord2[0]], [coord1[1], coord2[1]],
                    color=color, linewidth=linewidth, alpha=0.7)

            # Add force labels for significant forces
            if abs(force) > max_force * 0.1:
                mid_x = (coord1[0] + coord2[0]) / 2
                mid_y = (coord1[1] + coord2[1]) / 2
                ax3.text(mid_x, mid_y, f'{force:.1f}', ha='center', va='center',
                        bbox=dict(boxstyle="round,pad=0.1", facecolor='white', alpha=0.7))
        except:
            continue

    # Add nodes
    for node in all_nodes:
        coord = ops.nodeCoord(node)
        ax3.plot(coord[0], coord[1], 'ko', markersize=6)

    ax3.axis('equal')
    ax3.text(0.02, 0.98, 'Red: Tension\nBlue: Compression',
             transform=ax3.transAxes, va='top',
             bbox=dict(boxstyle="round", facecolor='lightgray', alpha=0.7))

    # Plot 4: Load diagram
    ax4.set_title('Applied Loads')
    ax4.grid(True, alpha=0.3)

    for node in all_nodes:
        coord = ops.nodeCoord(node)
        ax4.plot(coord[0], coord[1], 'ko', markersize=6)

        if node in loaded_nodes:
            # Draw load arrow
            ax4.arrow(coord[0], coord[1], 0, -1, head_width=0.5, head_length=0.3,
                     fc='red', ec='red', linewidth=2)
            ax4.text(coord[0], coord[1]-1.5, f'{load_magnitude/1000:.0f}kN',
                    ha='center', va='top', color='red', fontweight='bold')

    # Draw structure
    for elem in range(1, element_tag):
        try:
            nodes = ops.eleNodes(elem)
            coord1 = ops.nodeCoord(nodes[0])
            coord2 = ops.nodeCoord(nodes[1])
            ax4.plot([coord1[0], coord2[0]], [coord1[1], coord2[1]], 'k-', linewidth=1, alpha=0.5)
        except:
            continue

    ax4.axis('equal')

    plt.tight_layout()
    plt.show()

# Uncomment to plot results
# plot_results()

print(f"\nModel Summary:")
print(f"- Total nodes: {len(all_nodes)}")
print(f"- Total elements: {element_tag-1}")
print(f"- Applied load: {len(loaded_nodes) * load_magnitude/1000:.0f} kN total")
print(f"- Analysis status: {'SUCCESS' if success else 'FAILED'}")

Creating corrected truss model based on the geometry...
Creating elements...
Bottom chord elements:
  Element 1: Node 1 to Node 3
  Element 2: Node 3 to Node 5
  Element 3: Node 5 to Node 7
  Element 4: Node 7 to Node 9
  Element 5: Node 9 to Node 11
Top chord elements:
  Element 6: Node 2 to Node 4
  Element 7: Node 4 to Node 6
  Element 8: Node 6 to Node 8
  Element 9: Node 8 to Node 10
Web elements:
  Element 10: Node 1 to Node 2
  Element 11: Node 2 to Node 3
  Element 12: Node 3 to Node 4
  Element 13: Node 4 to Node 5
  Element 14: Node 5 to Node 6
  Element 15: Node 6 to Node 7
  Element 16: Node 7 to Node 8
  Element 17: Node 8 to Node 9
  Element 18: Node 9 to Node 10
  Element 19: Node 10 to Node 11
  Element 20: Node 1 to Node 4
  Element 21: Node 3 to Node 6
  Element 22: Node 5 to Node 8
  Element 23: Node 7 to Node 10

Applying boundary conditions...
  Node 1: Pinned support (fixed in X and Y)
  Node 11: Roller support (fixed in Y, free in X)

Applying loads (-10 kN downw

In [7]:
import openseespy.opensees as ops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import json

print("="*60)
print("STRUCTURAL HEALTH MONITORING DATA GENERATION")
print("AI-Based Damage Detection Using Modal Frequencies")
print("="*60)

def create_truss_model():
    """Create the baseline truss model"""
    ops.wipe()
    ops.model('basic', '-ndm', 2, '-ndf', 2)

    # Define nodes (same as corrected model)
    ops.node(1, 0.0, 0.0)
    ops.node(3, 6.0, 0.0)
    ops.node(5, 12.0, 0.0)
    ops.node(7, 18.0, 0.0)
    ops.node(9, 24.0, 0.0)
    ops.node(11, 30.0, 0.0)

    height = 4.5
    ops.node(2, 3.0, height)
    ops.node(4, 9.0, height)
    ops.node(6, 15.0, height)
    ops.node(8, 21.0, height)
    ops.node(10, 27.0, height)

    # Material properties
    E = 200000.0e6  # Pa
    A_chord = 0.01  # m²
    A_web = 0.005   # m²

    ops.uniaxialMaterial('Elastic', 1, E)

    return A_chord, A_web

def create_elements(A_chord, A_web, damage_element=None, damage_percentage=0.0):
    """Create elements with optional damage simulation"""
    element_tag = 1
    element_info = {}

    # Bottom chord elements
    bottom_connections = [(1,3), (3,5), (5,7), (7,9), (9,11)]
    for connection in bottom_connections:
        area = A_chord
        if damage_element == element_tag:
            area *= (1 - damage_percentage/100.0)  # Reduce area to simulate damage
        ops.element('Truss', element_tag, connection[0], connection[1], area, 1)
        element_info[element_tag] = {
            'nodes': connection,
            'type': 'bottom_chord',
            'original_area': A_chord,
            'actual_area': area
        }
        element_tag += 1

    # Top chord elements
    top_connections = [(2,4), (4,6), (6,8), (8,10)]
    for connection in top_connections:
        area = A_chord
        if damage_element == element_tag:
            area *= (1 - damage_percentage/100.0)
        ops.element('Truss', element_tag, connection[0], connection[1], area, 1)
        element_info[element_tag] = {
            'nodes': connection,
            'type': 'top_chord',
            'original_area': A_chord,
            'actual_area': area
        }
        element_tag += 1

    # Web elements
    web_connections = [
        (1, 2), (2, 3), (3, 4), (4, 5), (5, 6),
        (6, 7), (7, 8), (8, 9), (9, 10), (10, 11)
    ]
    for connection in web_connections:
        area = A_web
        if damage_element == element_tag:
            area *= (1 - damage_percentage/100.0)
        ops.element('Truss', element_tag, connection[0], connection[1], area, 1)
        element_info[element_tag] = {
            'nodes': connection,
            'type': 'web_member',
            'original_area': A_web,
            'actual_area': area
        }
        element_tag += 1

    return element_tag - 1, element_info

def apply_boundary_and_loads():
    """Apply supports and loads"""
    # Boundary conditions
    ops.fix(1, 1, 1)  # Pin
    ops.fix(11, 0, 1)  # Roller

    # Add masses for modal analysis
    rho = 7850.0  # kg/m³
    node_mass = 100.0  # kg per node
    all_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    for node in all_nodes:
        ops.mass(node, node_mass, node_mass)

def extract_modal_data(num_modes=10):
    """Extract natural frequencies and mode shapes"""
    eigenvalues = ops.eigen(num_modes)

    if not eigenvalues or any(ev <= 0 for ev in eigenvalues):
        return None, None

    frequencies = []
    mode_shapes = {}
    all_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    for i, eigenval in enumerate(eigenvalues):
        omega = eigenval**0.5
        frequency = omega / (2 * np.pi)
        frequencies.append(frequency)

        # Get mode shapes for each node
        mode_shapes[i+1] = {}
        for node in all_nodes:
            try:
                shape = ops.nodeEigenvector(node, i+1)
                mode_shapes[i+1][node] = {
                    'x_shape': shape[0],
                    'y_shape': shape[1]
                }
            except:
                mode_shapes[i+1][node] = {'x_shape': 0.0, 'y_shape': 0.0}

    return frequencies, mode_shapes

def generate_damage_dataset():
    """Generate comprehensive dataset for AI training"""
    print("Generating comprehensive dataset for AI-based damage detection...")

    dataset = []
    all_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    # First, get baseline (healthy) structure data
    print("Analyzing healthy structure...")
    A_chord, A_web = create_truss_model()
    total_elements, element_info = create_elements(A_chord, A_web)
    apply_boundary_and_loads()

    # Setup analysis
    ops.constraints('Transformation')
    ops.numberer('RCM')
    ops.system('BandGeneral')

    baseline_freq, baseline_modes = extract_modal_data(num_modes=6)

    if baseline_freq is None:
        print("❌ Failed to extract baseline frequencies")
        return None

    # Add baseline data
    baseline_row = {
        'damage_element': 0,  # 0 means no damage
        'damage_percentage': 0.0,
        'case_description': 'Healthy Structure'
    }

    # Add frequencies
    for i, freq in enumerate(baseline_freq):
        baseline_row[f'frequency_{i+1}'] = freq

    # Add mode shapes at each node
    for mode in range(1, len(baseline_freq)+1):
        for node in all_nodes:
            baseline_row[f'mode_{mode}_node_{node}_x'] = baseline_modes[mode][node]['x_shape']
            baseline_row[f'mode_{mode}_node_{node}_y'] = baseline_modes[mode][node]['y_shape']

    dataset.append(baseline_row)
    print(f"✓ Baseline frequencies: {[f'{f:.3f}' for f in baseline_freq]} Hz")

    # Generate damage scenarios
    damage_percentages = [1, 2, 5, 10, 15, 20]  # Different damage levels

    for element_num in range(1, total_elements + 1):
        for damage_pct in damage_percentages:
            print(f"Analyzing damage: Element {element_num}, {damage_pct}% reduction...")

            # Create model with damage
            ops.wipe()
            A_chord, A_web = create_truss_model()
            _, element_info = create_elements(A_chord, A_web,
                                            damage_element=element_num,
                                            damage_percentage=damage_pct)
            apply_boundary_and_loads()

            # Setup analysis
            ops.constraints('Transformation')
            ops.numberer('RCM')
            ops.system('BandGeneral')

            # Extract modal data
            damaged_freq, damaged_modes = extract_modal_data(num_modes=6)

            if damaged_freq is None:
                print(f"  ❌ Failed for element {element_num}")
                continue

            # Create data row
            data_row = {
                'damage_element': element_num,
                'damage_percentage': damage_pct,
                'case_description': f'Element {element_num} - {damage_pct}% damage'
            }

            # Add frequencies and frequency changes
            for i, (freq, base_freq) in enumerate(zip(damaged_freq, baseline_freq)):
                data_row[f'frequency_{i+1}'] = freq
                data_row[f'freq_change_{i+1}'] = freq - base_freq
                data_row[f'freq_change_pct_{i+1}'] = ((freq - base_freq) / base_freq) * 100

            # Add mode shapes
            for mode in range(1, len(damaged_freq)+1):
                for node in all_nodes:
                    data_row[f'mode_{mode}_node_{node}_x'] = damaged_modes[mode][node]['x_shape']
                    data_row[f'mode_{mode}_node_{node}_y'] = damaged_modes[mode][node]['y_shape']

                    # Add mode shape changes
                    baseline_x = baseline_modes[mode][node]['x_shape']
                    baseline_y = baseline_modes[mode][node]['y_shape']
                    data_row[f'mode_change_{mode}_node_{node}_x'] = damaged_modes[mode][node]['x_shape'] - baseline_x
                    data_row[f'mode_change_{mode}_node_{node}_y'] = damaged_modes[mode][node]['y_shape'] - baseline_y

            dataset.append(data_row)
            print(f"  ✓ Frequency changes: {[f'{f-b:.6f}' for f, b in zip(damaged_freq, baseline_freq)]}")

    return dataset, element_info, baseline_freq

def save_dataset(dataset, element_info, baseline_freq):
    """Save dataset in various formats"""
    if not dataset:
        print("No dataset to save!")
        return

    # Convert to DataFrame
    df = pd.DataFrame(dataset)

    # Save as CSV
    csv_filename = 'structural_health_monitoring_dataset.csv'
    df.to_csv(csv_filename, index=False)
    print(f"✓ Dataset saved as CSV: {csv_filename}")
    print(f"  - Total samples: {len(df)}")
    print(f"  - Features per sample: {len(df.columns)}")

    # Save element information
    with open('element_info.json', 'w') as f:
        json.dump(element_info, f, indent=2)
    print("✓ Element information saved: element_info.json")

    # Save baseline frequencies
    baseline_info = {
        'baseline_frequencies': baseline_freq,
        'frequency_labels': [f'Mode {i+1}' for i in range(len(baseline_freq))]
    }
    with open('baseline_frequencies.json', 'w') as f:
        json.dump(baseline_info, f, indent=2)
    print("✓ Baseline frequencies saved: baseline_frequencies.json")

    return df

def analyze_dataset(df):
    """Analyze the generated dataset"""
    print("\n" + "="*50)
    print("DATASET ANALYSIS")
    print("="*50)

    # Basic statistics
    print(f"Total samples: {len(df)}")
    print(f"Features per sample: {len(df.columns)}")
    print(f"Unique damage elements: {df['damage_element'].nunique()}")
    print(f"Damage levels: {sorted(df['damage_percentage'].unique())}")

    # Frequency analysis
    freq_columns = [col for col in df.columns if col.startswith('frequency_')]
    print(f"\nFrequency features: {len(freq_columns)}")

    print("\nFrequency ranges:")
    for col in freq_columns:
        print(f"  {col}: {df[col].min():.3f} - {df[col].max():.3f} Hz")

    # Frequency change analysis
    change_columns = [col for col in df.columns if col.startswith('freq_change_') and not col.endswith('_pct')]
    print(f"\nMaximum frequency changes:")
    for col in change_columns:
        max_change = df[col].abs().max()
        print(f"  {col}: ±{max_change:.6f} Hz")

    # Mode shape analysis
    mode_columns = [col for col in df.columns if col.startswith('mode_') and not col.startswith('mode_change_')]
    print(f"\nMode shape features: {len(mode_columns)}")

    return freq_columns, change_columns, mode_columns

def create_ai_ready_features(df):
    """Create features specifically for AI/ML algorithms"""
    print("\n" + "="*50)
    print("CREATING AI-READY FEATURE MATRIX")
    print("="*50)

    # Features for damage detection
    feature_columns = []

    # 1. Frequency changes (most important for damage detection)
    freq_change_cols = [col for col in df.columns if col.startswith('freq_change_') and not col.endswith('_pct')]
    feature_columns.extend(freq_change_cols)

    # 2. Frequency change percentages
    freq_change_pct_cols = [col for col in df.columns if col.startswith('freq_change_pct_')]
    feature_columns.extend(freq_change_pct_cols)

    # 3. Mode shape changes
    mode_change_cols = [col for col in df.columns if col.startswith('mode_change_')]
    feature_columns.extend(mode_change_cols)

    # Create feature matrix
    X = df[feature_columns].values
    y_element = df['damage_element'].values  # Target: which element is damaged
    y_severity = df['damage_percentage'].values  # Target: damage severity

    print(f"Feature matrix shape: {X.shape}")
    print(f"Features selected: {len(feature_columns)}")
    print(f"Samples: {X.shape[0]}")

    # Save feature matrix
    feature_data = {
        'features': X.tolist(),
        'target_element': y_element.tolist(),
        'target_severity': y_severity.tolist(),
        'feature_names': feature_columns
    }

    with open('ai_features.json', 'w') as f:
        json.dump(feature_data, f, indent=2)
    print("✓ AI-ready features saved: ai_features.json")

    return X, y_element, y_severity, feature_columns

def create_sample_ml_code():
    """Generate sample Python code for machine learning"""
    ml_code = '''
# Sample Machine Learning Code for Structural Health Monitoring
# Load this code in Google Colab or Jupyter Notebook

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('structural_health_monitoring_dataset.csv')
print(f"Dataset loaded: {df.shape}")

# Extract features for damage detection
feature_cols = [col for col in df.columns if col.startswith('freq_change_') and not col.endswith('_pct')]
X = df[feature_cols].values
y = df['damage_element'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Method 1: K-Means Clustering (Unsupervised)
print("\\n" + "="*50)
print("K-MEANS CLUSTERING APPROACH")
print("="*50)

n_clusters = len(np.unique(y))  # Number of elements + 1 for healthy
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_train_scaled)

# Method 2: Random Forest Classifier (Supervised)
print("\\n" + "="*50)
print("RANDOM FOREST CLASSIFICATION")
print("="*50)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf_classifier.predict(X_test_scaled)

# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# Visualizations
plt.figure(figsize=(15, 5))

# Plot 1: Feature importance
plt.subplot(1, 3, 1)
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Top 10 Feature Importance')

# Plot 2: Confusion matrix
plt.subplot(1, 3, 2)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
plt.title('Confusion Matrix')

# Plot 3: Frequency changes vs damage element
plt.subplot(1, 3, 3)
plt.scatter(df['freq_change_1'], df['freq_change_2'], c=df['damage_element'], cmap='viridis')
plt.xlabel('Mode 1 Frequency Change (Hz)')
plt.ylabel('Mode 2 Frequency Change (Hz)')
plt.title('Frequency Changes by Damage Location')
plt.colorbar(label='Damage Element')

plt.tight_layout()
plt.show()

# Function to predict damage for new data
def predict_damage(frequency_changes):
    """
    Predict damaged element from frequency changes
    frequency_changes: list of frequency changes for each mode
    """
    # Scale the input
    freq_array = np.array(frequency_changes).reshape(1, -1)
    freq_scaled = scaler.transform(freq_array)

    # Predict
    prediction = rf_classifier.predict(freq_scaled)[0]
    probability = rf_classifier.predict_proba(freq_scaled)[0].max()

    return prediction, probability

# Example prediction
example_freq_changes = [-0.001, 0.002, -0.0005, 0.001, 0.0008, -0.0003]
predicted_element, confidence = predict_damage(example_freq_changes)
print(f"\\nExample Prediction:")
print(f"Frequency changes: {example_freq_changes}")
print(f"Predicted damaged element: {predicted_element}")
print(f"Confidence: {confidence:.3f}")
'''

    with open('sample_ml_analysis.py', 'w') as f:
        f.write(ml_code)
    print("✓ Sample ML code saved: sample_ml_analysis.py")

# Main execution
if __name__ == "__main__":
    # Generate the dataset
    dataset, element_info, baseline_freq = generate_damage_dataset()

    if dataset:
        # Save dataset
        df = save_dataset(dataset, element_info, baseline_freq)

        # Analyze dataset
        freq_cols, change_cols, mode_cols = analyze_dataset(df)

        # Create AI-ready features
        X, y_element, y_severity, feature_names = create_ai_ready_features(df)

        # Create sample ML code
        create_sample_ml_code()

        print("\n" + "="*60)
        print("DATASET GENERATION COMPLETE!")
        print("="*60)
        print("Files created:")
        print("1. structural_health_monitoring_dataset.csv - Main dataset")
        print("2. element_info.json - Element information")
        print("3. baseline_frequencies.json - Healthy structure frequencies")
        print("4. ai_features.json - AI-ready feature matrix")
        print("5. sample_ml_analysis.py - Sample ML code")

        print(f"\nDataset Summary:")
        print(f"- Total scenarios: {len(df)}")
        print(f"- Elements analyzed: {len(element_info)}")
        print(f"- Damage levels: 1%, 2%, 5%, 10%, 15%, 20%")
        print(f"- Features per sample: {len(feature_names)}")
        print(f"- Ready for AI/ML analysis!")

        # Show sample data
        print(f"\nSample of frequency changes (first 5 damaged cases):")
        damaged_cases = df[df['damage_element'] != 0].head(5)
        for idx, row in damaged_cases.iterrows():
            print(f"Element {row['damage_element']}, {row['damage_percentage']}% damage:")
            changes = [row[f'freq_change_{i+1}'] for i in range(6)]
            print(f"  Freq changes: {[f'{c:.6f}' for c in changes]} Hz")

    else:
        print("❌ Dataset generation failed!")

STRUCTURAL HEALTH MONITORING DATA GENERATION
AI-Based Damage Detection Using Modal Frequencies
Generating comprehensive dataset for AI-based damage detection...
Analyzing healthy structure...
✓ Baseline frequencies: ['32.531', '70.745', '93.667', '147.925', '192.930', '202.099'] Hz
Analyzing damage: Element 1, 1% reduction...
  ✓ Frequency changes: ['-0.006320', '-0.140996', '-0.002227', '-0.007754', '-0.063306', '-0.002209']
Analyzing damage: Element 1, 2% reduction...
  ✓ Frequency changes: ['-0.012780', '-0.283978', '-0.004458', '-0.015609', '-0.127528', '-0.004421']
Analyzing damage: Element 1, 5% reduction...
  ✓ Frequency changes: ['-0.033049', '-0.725257', '-0.011176', '-0.039798', '-0.325876', '-0.011078']
Analyzing damage: Element 1, 10% reduction...
  ✓ Frequency changes: ['-0.070115', '-1.504487', '-0.022461', '-0.082320', '-0.676525', '-0.022236']
Analyzing damage: Element 1, 15% reduction...
  ✓ Frequency changes: ['-0.111974', '-2.343704', '-0.033856', '-0.127851', '-1.05